In [1]:
!conda config --add pkgs_dirs ./ # Set the location where conda package will be downloaded
!conda install --download-only -y pyvips # Download pyvips and dependencies

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ 

In [2]:
!conda install *.tar.bz2 


######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
###########################

In [3]:
# import pandas as pd
# from collections import Counter


# data = pd.read_csv('/kaggle/input/mayo-clinic-strip-ai/train.csv')
# res = dict(Counter(data['center_id'].tolist()))
# print(res)

In [4]:
# groups = [(11,), (4,), (7,), (1, 5,), (10, 3), (6, 2, 8, 9,)]
# for gr in groups:
#     print(gr, sum(res[i] for i in gr))

In [5]:
BAD_IMAGE_IDS = ['5adc4c_0', '7b9aaa_0', 'bb06a5_0', 'e26a04_0', '280c26_0'] + \
                ['4ae44b_0', '53e66f_0', '7c2c2f_0', '74a450_1']

BLOCK_SIZE = 28
BLOCKS_PER_CROP = 8
CROP_SIZE = BLOCK_SIZE * BLOCKS_PER_CROP
BLOCK_THR = 90
CROP_THR = 0.6
MAX_CROPS_PER_IMAGE = 20
IMAGES_PER_SAMPLE = 4
EPOCHS_NUM = 5
SCALE_FACTOR = 24

In [6]:
import gc
import os
from time import time
from typing import List, Tuple

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import pyvips
import cv2


class DataPreparation:
    def __init__(self, visualize: bool = False, seed: int = 42):
        self.visualize = visualize
        self.seed = seed

        train_metadata = pd.read_csv('/kaggle/input/mayo-clinic-strip-ai/train.csv')
        train_metadata = list(zip(
            train_metadata['image_id'].tolist(),
            train_metadata['label'].tolist(),
            train_metadata['center_id'].tolist(),
        ))
        self.train = self._filter_bad_images(train_metadata)
        self.all_center_ids = sorted(list({center_id for _, _, center_id in self.train}))

        other_metadata = pd.read_csv('/kaggle/input/mayo-clinic-strip-ai/other.csv').query('label == \'Other\'')
        other_metadata = list(zip(
            other_metadata['image_id'].tolist(),
            ['LAA' for _ in range(other_metadata.shape[0])],
            [-1 for _ in range(other_metadata.shape[0])],
        ))
        self.other = self._filter_bad_images(other_metadata)

    @staticmethod
    def _filter_bad_images(data: List[Tuple]) -> List[Tuple]:
        return [
            (image_id, label, center_id)
            for image_id, label, center_id in data
            if image_id not in BAD_IMAGE_IDS
        ]

    @staticmethod
    def _add_rect_to_numpy(image: np.ndarray, x: int, y: int, size: int, thickness: int) -> None:
        image[x:x + size, y:y + thickness] = (0, 0, 0)
        image[x:x + thickness, y:y + size] = (0, 0, 0)
        image[x:x + size, y + size:y + size + thickness] = (0, 0, 0)
        image[x + size:x + size + thickness, y:y + size] = (0, 0, 0)

    @staticmethod
    def _get_blocks_map(image: np.ndarray) -> np.ndarray:
        pixels_diff = np.sum((image[:-1, :, :] - image[1:, :, :]) ** 2, axis=2)
        pixels_diff = np.cumsum(np.cumsum(pixels_diff, axis=0), axis=1)
        blocks_map = np.zeros((
            (image.shape[0] + BLOCK_SIZE - 1) // BLOCK_SIZE,
            (image.shape[1] + BLOCK_SIZE - 1) // BLOCK_SIZE,
        ))
        for x in range(0, pixels_diff.shape[0], BLOCK_SIZE):
            for y in range(0, pixels_diff.shape[1], BLOCK_SIZE):
                nx = min(x + BLOCK_SIZE, pixels_diff.shape[0])
                ny = min(y + BLOCK_SIZE, pixels_diff.shape[1])
                block_sum = int(pixels_diff[nx - 1, ny - 1])
                if x:
                    block_sum -= int(pixels_diff[x - 1, ny - 1])
                if y:
                    block_sum -= int(pixels_diff[nx - 1, y - 1])
                if x and y:
                    block_sum += int(pixels_diff[x - 1, y - 1])
                blocks_map[x // BLOCK_SIZE][y // BLOCK_SIZE] = \
                    (block_sum / BLOCK_SIZE / BLOCK_SIZE) > BLOCK_THR
        return blocks_map

    def _generate_crops_positions(
            self,
            image: np.ndarray,
            crop_thr: float,
    ) -> Tuple[List[Tuple[int, int]], np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        blocks_map = self._get_blocks_map(image)

        if self.visualize:
            for i in range(blocks_map.shape[0]):
                for j in range(blocks_map.shape[1]):
                    if blocks_map[i][j]:
                        self._add_rect_to_numpy(
                            image,
                            i * BLOCK_SIZE,
                            j * BLOCK_SIZE,
                            BLOCK_SIZE,
                            1,
                        )

        good_crops_starts = []
        for x in range(0, image.shape[0] - CROP_SIZE + 1, BLOCK_SIZE):
            for y in range(0, image.shape[1] - CROP_SIZE + 1, BLOCK_SIZE):
                _x, _y = x // BLOCK_SIZE, y // BLOCK_SIZE
                crop_sum = blocks_map[_x:_x + BLOCKS_PER_CROP, _y:_y + BLOCKS_PER_CROP].sum()
                if crop_sum > BLOCKS_PER_CROP * BLOCKS_PER_CROP * crop_thr:
                    good_crops_starts.append((x, y))

        if self.visualize:
            for x, y in good_crops_starts:
                self._add_rect_to_numpy(image, x, y, CROP_SIZE, 1)

        return good_crops_starts

    @staticmethod
    def _process_crop(crop: np.ndarray) -> np.ndarray:
        return crop

    def _create_crops(
        self,
        image: np.ndarray,
        crops_starts: List[Tuple[int]],
    ) -> List[np.ndarray]:
        return [
            Image.fromarray(
                self._process_crop(
                    image[x:x + CROP_SIZE, y:y + CROP_SIZE],
                )
            )
            for x, y in crops_starts
        ]

    @staticmethod
    def _get_unique_crops(crop_starts: List[Tuple[int, int]], order) -> List[Tuple[int, int]]:
        def inter_size_1d(a: int, b: int, c: int, d: int) -> int:
            return max(0, min(b, d) - max(a, c))

        def inter_size_2d(crop_start_1: Tuple[int, int], crop_start_2: Tuple[int, int]) -> int:
            return inter_size_1d(
                crop_start_1[0], crop_start_1[0] + CROP_SIZE,
                crop_start_2[0], crop_start_2[0] + CROP_SIZE,
            ) * inter_size_1d(
                crop_start_1[1], crop_start_1[1] + CROP_SIZE,
                crop_start_2[1], crop_start_2[1] + CROP_SIZE,
            )

        crop_starts_sorted = sorted(crop_starts, key=order)
        final_crop_starts = []
        for crop_start in crop_starts_sorted:
            if any(
                    inter_size_2d(crop_start, crop_start_prev) > CROP_SIZE * CROP_SIZE // 2
                    for crop_start_prev in final_crop_starts
            ):
                continue
            final_crop_starts.append(crop_start)
        return final_crop_starts
    
    @staticmethod
    def _read_and_resize_image(image_id: str, base_image_path: str) -> np.ndarray:       
        image_path = os.path.join(base_image_path, f'{image_id}.tif')
        image = pyvips.Image.new_from_file(image_path, access='sequential')
        return image.resize(1.0 / SCALE_FACTOR).numpy()
    
    def prepare_crops(
            self,
            image_ids: List[int],
            base_image_path: str,
    ) -> Tuple[List[List[np.ndarray]], List[List[Tuple[int]]], List[Tuple[np.ndarray, np.ndarray]]]:
        np.random.seed(self.seed)
        image_crops = []
        image_crops_indices = []
        for image_id in tqdm(image_ids):
            start_time = time()
            image = self._read_and_resize_image(image_id, base_image_path)
            gc.collect()
            print(f'Rescaling done in {time() - start_time} seconds. Image shape is {image.shape}')
            found_flag = False
            for crop_thr in np.arange(CROP_THR, -0.1, -0.1):
                good_crops_starts = self._generate_crops_positions(image, crop_thr)
                if len(good_crops_starts) < IMAGES_PER_SAMPLE:
                    print('Bad image', image_id, 'crop_thr', crop_thr, 'only', len(good_crops_starts))
                    continue

                good_crops_starts_unique = []
                for order in [
                    lambda x: (x[0], x[1]),
                    lambda x: (-x[0], -x[1]),
                ]:
                    good_crops_starts_unique.extend(self._get_unique_crops(good_crops_starts, order))
                good_crops_starts_unique = list(set(good_crops_starts_unique))

                if len(good_crops_starts_unique) < IMAGES_PER_SAMPLE:
                    print('Bad image', image_id, 'crop_thr', crop_thr, 'only', len(good_crops_starts_unique))
                    continue

                good_crops_starts_sample_ids = np.random.choice(
                    list(range(len(good_crops_starts_unique))),
                    min(len(good_crops_starts_unique), MAX_CROPS_PER_IMAGE),
                    replace=False,
                )
                good_crops_starts_sample = np.array(good_crops_starts_unique)[good_crops_starts_sample_ids]
                image_crops_indices.append(good_crops_starts_sample)
                image_crops.append(self._create_crops(image, good_crops_starts_sample))
                found_flag = True
                break
            if not found_flag:
                image_crops_indices.append([])
                image_crops.append([])
                print('No crops was found')
            print(f'Done {image_id} in {time() - start_time} seconds')
        gc.collect()
        return image_crops, image_crops_indices

    def process_train(
            self
    ) -> Tuple[List[List[np.ndarray]], List[List[Tuple[int]]]]:
        return self.prepare_crops(
            [image_id for image_id, _, _ in self.train],
            '/kaggle/input/mayo-clinic-strip-ai/train/',
        )

    def process_other(
            self
    ) -> Tuple[List[List[np.ndarray]], List[List[Tuple[int]]]]:
        return self.prepare_crops(
            [image_id for image_id, _, _ in self.other],
            '/kaggle/input/mayo-clinic-strip-ai/other/',
        )

In [7]:
import random
from collections import defaultdict
from typing import List

import numpy as np
import torch
from PIL import Image


class ClotImageDataset(torch.utils.data.Dataset):
    def __init__(
            self,
            image_ids: List[str],
            labels: List[str],
            image_crops: List[List[np.ndarray]],
            seed: int,
            is_test: bool,
            transformations,
    ):
        self.image_ids = image_ids
        self.labels = [float(label == 'CE') for label in labels]
        self.image_crops = image_crops
        self.seed = seed
        self.is_test = is_test
        self.transformations = transformations

        if not self.is_test:
            np.random.seed(self.seed)

            label_to_indices = defaultdict(list)
            for i, (label, crops) in enumerate(zip(self.labels, self.image_crops)):
                if len(crops) > 0:
                    label_to_indices[label].append(i)

            max_size = 4 * max(len(indices) for indices in label_to_indices.values())

            self.sample_ids = []
            for i, indices in enumerate(label_to_indices.values()):
                np.random.shuffle(indices)
                while len(self.sample_ids) < max_size * (i + 1):
                    req_size = min(len(indices), max_size * (i + 1) - len(self.sample_ids))
                    self.sample_ids += indices[:req_size]
        else:
            self.sample_ids = []
            for _ in range(20):
                self.sample_ids.extend(list(range(len(self.image_ids))))

        self.image_index_ids = []
        sample_id_to_image_index = defaultdict(int)
        for sample_id in self.sample_ids:
            self.image_index_ids.append(sample_id_to_image_index[sample_id])
            image_crops_cnt = len(self.image_crops[sample_id])
            if image_crops_cnt:
                sample_id_to_image_index[sample_id] = (sample_id_to_image_index[sample_id] + 1) % image_crops_cnt

    def __len__(self):
        return len(self.sample_ids)

    def __getitem__(self, idx):
        if self.is_test:
            np.random.seed(self.seed + idx)
            random.seed(self.seed + idx)
            torch.manual_seed(self.seed + idx)
        idx, image_index = self.sample_ids[idx], self.image_index_ids[idx]
        if len(self.image_crops[idx]) == 0:
            return (
                self.transformations(Image.fromarray(np.zeros((224, 224, 3)).astype(np.uint8))),
                torch.tensor(self.labels[idx]),
                self.image_ids[idx],
            )
        # image_index = np.random.randint(0, len(self.image_crops[idx]))
        return (
            self.transformations(self.image_crops[idx][image_index]),
            torch.tensor(self.labels[idx]),
            self.image_ids[idx],
        )


def get_loader(
        image_ids: List[str],
        labels: List[str],
        image_crops: List[List[np.ndarray]],
        seed: int,
        is_test: bool,
        transformations,
        shuffle: bool,
        batch_size: int,
        num_workers: int
):
    dataset = ClotImageDataset(
        image_ids, labels, image_crops, seed, is_test, transformations,
    )
    return torch.utils.data.DataLoader(dataset, shuffle=shuffle, batch_size=batch_size, num_workers=num_workers)

In [8]:
from collections import defaultdict

import numpy as np


def get_target_metric(y_true, y_pred, image_ids):
    patients = [image_id.split('_')[0] for image_id in image_ids]
    patient_to_y_true, patient_to_y_pred = defaultdict(list), defaultdict(list)
    for y, y_hat, patient in zip(y_true, y_pred, patients):
        patient_to_y_true[patient].append(y)
        patient_to_y_pred[patient].append(y_hat)
    patient_to_y_true = {
        patient: np.mean(y_true)
        for patient, y_true in patient_to_y_true.items()
    }
    patient_to_y_pred = {
        patient: np.mean(y_pred).tolist()
        for patient, y_pred in patient_to_y_pred.items()
    }
    y_true, y_pred = [], []
    for patient, y in patient_to_y_true.items():
        y_true.append(y)
        y_pred.append(patient_to_y_pred[patient])
    return _weighted_mc_log_loss(y_true, np.array([[1 - p, p] for p in y_pred]))


def _weighted_mc_log_loss(y_true, y_pred, epsilon=1e-15):
    class_cnt = [sum(int(val == cl) for val in y_true) for cl in range(2)]
    w = [0.5 for _ in range(2)]
    return -sum(
        w[cl] * sum(
            (y == cl) / class_cnt[cl] * np.log(max(min(y_hat, 1 - epsilon), epsilon))
            for y, y_hat in zip(y_true, y_pred[:, cl])
        )
        for cl in range(2)
    ) / sum(w[cl] for cl in range(2))

In [9]:
import torch
import torch.nn as nn
# import pretrainedmodels as pm
from torchvision import models


class ClotModelMIL(nn.Module):
    def __init__(self, num_crops=None):
        super().__init__()
        self.num_crops = num_crops

        base_model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        self.model = nn.Sequential(*list(base_model.children())[:-2])
        in_features_cnt = list(base_model.children())[-1].in_features
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(output_size=1),
            nn.Flatten(),
            nn.Linear(in_features_cnt, 1),
            nn.Sigmoid(),
        )

    def freeze_encoder(self, flag):
        for param in self.model.parameters():
            param.requires_grad = not flag

    def forward(self, x):
        # x: bs x N x C x W x W
        bs, _, ch, w, h = x.shape
        x = x.view(bs * self.num_crops, ch, w, h)  # x: N bs x C x W x W
        x = self.model(x)  # x: N bs x C' x W' x W'

        # Concat and pool
        bs2, ch2, w2, h2 = x.shape
        x = x \
            .view(-1, self.num_crops, ch2, w2, h2) \
            .permute(0, 2, 1, 3, 4) \
            .contiguous() \
            .view(bs, ch2, self.num_crops * w2, h2)  # x: bs x C' x N W'' x W''
        return self.head(x)

    def save(self, model_path):
        weights = self.state_dict()
        torch.save(weights, model_path)

    def load(self, model_path):
        weights = torch.load(model_path, map_location='cpu')
        self.load_state_dict(weights)


class ClotModelSingle(nn.Module):
    def __init__(self, encoder_model):
        super().__init__()

        if encoder_model == 'effnet_b0':
            base_model = models.efficientnet_b0(pretrained=True) # weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
            self.model = base_model.features
            in_features_cnt = base_model.classifier[1].in_features
        elif encoder_model == 'resnet18':
            base_model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
            self.model = nn.Sequential(*list(base_model.children())[:-2])
            in_features_cnt = list(base_model.children())[-1].in_features
        elif encoder_model == 'regnet_x_1_6gf':
            base_model = models.regnet_x_1_6gf(weights=models.RegNet_X_1_6GF_Weights.IMAGENET1K_V2)
            self.model = nn.Sequential(base_model.stem, base_model.trunk_output)
            in_features_cnt = base_model.fc.in_features
        else:
            raise Exception('Incorrect encoder name')

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(output_size=1),
            nn.Flatten(),
            nn.Linear(in_features_cnt, 1),
            nn.Sigmoid(),
        )

    def freeze_encoder(self, flag):
        for param in self.model.parameters():
            param.requires_grad = not flag

    def forward(self, x):
        return self.head(self.model(x))

    def save(self, model_path):
        weights = self.state_dict()
        torch.save(weights, model_path)

    def load(self, model_path):
        weights = torch.load(model_path, map_location='cpu')
        self.load_state_dict(weights)

In [10]:
import os


os.makedirs('/kaggle/working/models/', exist_ok=True)

In [11]:
from __future__ import print_function, division

import os
import pickle
import sys
from collections import Counter

import cv2
import numpy as np
import ssl
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from PIL import Image
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
from torchvision import transforms


ssl._create_default_https_context = ssl._create_unverified_context
cudnn.benchmark = True


DUMPED_DATALOADER_PATH = '/kaggle/input/track-4-dataprep/data_loaders.pkl'
DUMPED_DATALOADER_OTHER_PATH = '/kaggle/input/track-4-dataprep/data_loaders_other.pkl'


def get_sub_data(data, image_crops, image_crops_indices, sample_ids):
    return [data[i][0] for i in sample_ids], \
        [data[i][1] for i in sample_ids], \
        [data[i][2] for i in sample_ids], \
        [image_crops[i] for i in sample_ids], \
        [image_crops_indices[i] for i in sample_ids]


data_prep = DataPreparation()

# image_crops, image_crops_indices = data_prep.process_train()
# with open(DUMPED_DATALOADER_PATH, 'wb') as file:
#     pickle.dump([image_crops, image_crops_indices], file)

# image_crops_other, image_crops_indices_other = data_prep.process_other()
# with open(DUMPED_DATALOADER_OTHER_PATH, 'wb') as file:
#     pickle.dump([image_crops_other, image_crops_indices_other], file)
    
with open(DUMPED_DATALOADER_PATH, 'rb') as file:
    image_crops, image_crops_indices = pickle.load(file)
with open(DUMPED_DATALOADER_OTHER_PATH, 'rb') as file:
    image_crops_other, image_crops_indices_other = pickle.load(file)    

In [12]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        # transforms.RandomResizedCrop((224, 224), scale=(0.5, 1.0), ratio=(1.0, 1.0)),
        transforms.RandomAdjustSharpness(sharpness_factor=2, p=1.0),
        transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
        transforms.ColorJitter(brightness=0.2, saturation=0.5, hue=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        # transforms.RandomResizedCrop((224, 224), scale=(0.5, 1.0), ratio=(1.0, 1.0)),
        transforms.RandomAdjustSharpness(sharpness_factor=2, p=1.0),
        transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
        transforms.ColorJitter(brightness=0.2, saturation=0.5, hue=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_data = data_prep.train + data_prep.other
image_crops += image_crops_other
image_crops_indices += image_crops_indices_other

all_metrics = []
all_y, all_y_hat, all_image_ids = [], [], []
#for test_center_id in {center_id for _, _, center_id in train_data if center_id != -1}:
for test_centers_group in [(11,), (4,), (7,), (1, 5,), (10, 3), (6, 2, 8, 9,)]:
    best_validation_metric = None
    best_y, best_y_hat, best_image_ids = [], [], []
    for iteration in range(3):
        for _ in range(3):
            print('-' * 80)
        test_centers_group_str = '.'.join(map(str, test_centers_group))
        print(f'CV with {test_centers_group_str} as test')
        train_sample_ids = [i for i, (_, _, center_id) in enumerate(train_data) if center_id not in test_centers_group]
        test_sample_ids = [i for i, (_, _, center_id) in enumerate(train_data) if center_id in test_centers_group]

        train_image_ids, train_labels, train_center_ids, train_crops, train_crop_indices = get_sub_data(
            train_data,
            image_crops,
            image_crops_indices,
            train_sample_ids
        )
        test_image_ids, test_labels, test_center_ids, test_crops, test_crop_indices = get_sub_data(
            train_data,
            image_crops,
            image_crops_indices,
            test_sample_ids
        )
        print(f'Train/Test sizes: {len(train_labels)}/{len(test_labels)}')
        print('Train/Test label distribution:')
        print({key: value / len(train_labels) for key, value in dict(Counter(train_labels)).items()})
        print({key: value / len(test_labels) for key, value in dict(Counter(test_labels)).items()})

        dataloaders = {
            'train': get_loader(
                train_image_ids,
                train_labels,
                train_crops,
                seed=42,
                is_test=False,
                transformations=data_transforms['train'],
                shuffle=True,
                batch_size=256,
                num_workers=2,
            ),
            'test': get_loader(
                test_image_ids,
                test_labels,
                test_crops,
                seed=42,
                is_test=True,
                transformations=data_transforms['test'],
                shuffle=False,
                batch_size=256,
                num_workers=2,
            ),
        }

        model = ClotModelSingle(encoder_model='effnet_b0').to(device)
        model.freeze_encoder(True)
        criterion = nn.BCELoss()
        optimizer = optim.Adam(model.head.parameters(), lr=0.05, weight_decay=0.0001)

        train_loss, val_loss = [], []
        for epoch in range(EPOCHS_NUM):
            np.random.seed(epoch)

            #if epoch == 10:
            #    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=5e-4)

            print('*' * 80)
            print("epoch {}/{}".format(epoch + 1, EPOCHS_NUM))

            model.train()
            running_loss, running_score = 0.0, 0.0
            y_hat, y, image_ids = [], [], []
            for image, label, image_id in tqdm(dataloaders['train']):
                image = image.to(device)
                label = label.to(device)
                optimizer.zero_grad()
                y_pred = model.forward(image).squeeze()
                loss = criterion(y_pred, label)
                running_loss += loss.item()
                loss.backward()
                optimizer.step()

                y_pred = y_pred.cpu().detach().numpy().tolist()
                label = label.cpu().detach().numpy().tolist()
                y_hat.extend(y_pred)
                y.extend(label)
                image_ids.extend(image_id)

                running_score += sum([int(int(y_hat > 0.5) == y) for y_hat, y in zip(y_pred, label)])

            print('Train:')
            print(Counter([int(p > 0.5) for p in y_hat]))
            print('ROC AUC metric:', roc_auc_score(y, y_hat))
            print('target metric:', get_target_metric(y, y_hat, image_ids))

            epoch_score = running_score / len(dataloaders['train'].dataset)
            epoch_loss = running_loss / len(dataloaders['train'])
            train_loss.append(epoch_loss)
            print("loss: {}, accuracy: {}".format(epoch_loss, epoch_score))

            with torch.no_grad():
                model.eval()
                running_loss, running_score = 0.0, 0.0
                y_hat, y, image_ids = [], [], []
                for image, label, image_id in tqdm(dataloaders['test']):
                    image = image.to(device)
                    label = label.to(device)
                    optimizer.zero_grad()
                    y_pred = model.forward(image).squeeze()
                    loss = criterion(y_pred, label)
                    running_loss += loss.item()

                    y_pred = y_pred.cpu().detach().numpy().tolist()
                    label = label.cpu().detach().numpy().tolist()
                    y_hat.extend(y_pred)
                    y.extend(label)
                    image_ids.extend(image_id)

                    running_score += sum([int(int(y_hat > 0.5) == y) for y_hat, y in zip(y_pred, label)])

                bad_image_ids = {
                    image_id
                    for image_id, crops in zip(test_image_ids, test_crops)
                    if len(crops) == 0 
                }
                y_hat_fixed = [
                    0.5 if image_id in bad_image_ids else p
                    for p, image_id in zip(y_hat, image_ids)
                ]

                print('Validation:')
                print(Counter([int(p > 0.5) for p in y_hat_fixed]))
                print('ROC AUC metric:', roc_auc_score(y, y_hat_fixed))

                target_metric = get_target_metric(y, y_hat, image_ids)
                print('target metric:', target_metric)
                target_metric = get_target_metric(y, y_hat_fixed, image_ids)
                print('target metric fixed:', target_metric)            
                if best_validation_metric is None or target_metric < best_validation_metric:
                    best_validation_metric = target_metric
                    best_y, best_y_hat, best_image_ids = y, y_hat_fixed, image_ids
                    torch.save(
                        model,
                        os.path.join(
                            '/kaggle/working/models',
                            f'center_id_{test_centers_group_str}_epoch_{epoch}_target_{round(target_metric, 3)}.h5',
                        ),
                    )

                epoch_score = running_score / len(dataloaders['test'].dataset)
                epoch_loss = running_loss / len(dataloaders['test'])
                val_loss.append(epoch_loss)
                print("loss: {}, accuracy: {}".format(epoch_loss, epoch_score))

    print(f'Best validation metric: {best_validation_metric}')
    all_metrics.append(best_validation_metric)
    all_y.extend(best_y)
    all_y_hat.extend(best_y_hat)
    all_image_ids.extend(best_image_ids)

print(all_metrics)
print(np.mean(all_metrics))
final_metric = get_target_metric(
    all_y,
    all_y_hat,
    all_image_ids,
)
np.save('/kaggle/working/all_y.npy', all_y)
np.save('/kaggle/working/all_y_hat.npy', all_y_hat)
np.save('/kaggle/working/all_image_ids.npy', all_image_ids)
print('Full validation metric:', final_metric)

--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
CV with 11 as test
Train/Test sizes: 560/254
Train/Test label distribution:
{'CE': 0.6553571428571429, 'LAA': 0.34464285714285714}
{'CE': 0.6968503937007874, 'LAA': 0.3031496062992126}


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

********************************************************************************
epoch 1/5


100%|██████████| 12/12 [00:24<00:00,  2.06s/it]


...

Validation:
Counter({0: 1238, 1: 742})
ROC AUC metric: 0.5118406593406594
target metric: 0.7343959576593952
target metric fixed: 0.7381227353187751
loss: 0.9951399564743042, accuracy: 0.44343434343434346
********************************************************************************
epoch 5/5


100%|██████████| 15/15 [00:27<00:00,  1.86s/it]


Train:
Counter({0: 1821, 1: 1819})
ROC AUC metric: 0.6591725033208549
target metric: 0.5841782871692399
loss: 0.7223800778388977, accuracy: 0.6063186813186813


100%|██████████| 8/8 [00:15<00:00,  1.88s/it]


Validation:
Counter({1: 1261, 0: 719})
ROC AUC metric: 0.5915506715506715
target metric: 0.6558949888155032
target metric fixed: 0.6613030546518286
loss: 0.7192556038498878, accuracy: 0.6388888888888888
Best validation metric: 0.6384588541986868
[0.642027776187675, 0.6156763463174704, 0.5805691482270264, 0.6510807591842562, 0.65766523668334, 0.6384588541986868]
0.6309130201330758
Full validation metric: 0.6356833352527698


In [13]:
# [0.653830656159886, 0.6238455839150563, 0.605778830856576, 0.6647636130357559, 0.6544317172083116, 0.6532475492188892]
# 0.6426496583990792
# Full validation metric: 0.6421152903205289


# [0.6455449287371509, 0.6218588657257058, 0.5979860325098842, 0.6420611577083772, 0.675211714053326, 0.6503222829996482]
# 0.6388308302890154
# Full validation metric: 0.6404691832962497

#64 5e3
# [0.6476080393945753, 0.6079157320890402, 0.6050800143556536, 0.6378247315988896, 0.6661450085369098, 0.6504032066047876]
# 0.635829455429976
# Full validation metric: 0.6342736651767453

#64 1e3
# Best validation metric: 0.6557763951171189
# [0.651014691249416, 0.6347137845717545, 0.6147587091636446, 0.6665997338703076, 0.6587241317996946, 0.6557763951171189]
# 0.6469312409619894
# Full validation metric: 0.6467345027007692

In [14]:
import os


model_files = [file_name for file_name in list(os.listdir('/kaggle/working/models')) if file_name.endswith('.h5')]
print(model_files[:5])

['center_id_6.2.8.9_epoch_3_target_0.673.h5', 'center_id_7_epoch_0_target_0.638.h5', 'center_id_6.2.8.9_epoch_2_target_0.68.h5', 'center_id_4_epoch_4_target_0.616.h5', 'center_id_1.5_epoch_4_target_0.679.h5']


In [15]:
from collections import defaultdict


centers_to_models = defaultdict(list)
for model_file in model_files:
    center_id = model_file.split('_')[2]
    epoch = int(model_file.split('_')[4])
    metric = float(model_file.split('_')[6][:-3])
    
    centers_to_models[center_id].append((metric, epoch, model_file))
    
center_id_to_best_model_file_name = {
    center_id: sorted(model_files, key=lambda x: (x[0], x[1]))[0][2]
    for center_id, model_files in centers_to_models.items()
}
print(center_id_to_best_model_file_name)

{'6.2.8.9': 'center_id_6.2.8.9_epoch_1_target_0.638.h5', '7': 'center_id_7_epoch_3_target_0.581.h5', '4': 'center_id_4_epoch_4_target_0.616.h5', '1.5': 'center_id_1.5_epoch_4_target_0.651.h5', '10.3': 'center_id_10.3_epoch_3_target_0.658.h5', '11': 'center_id_11_epoch_2_target_0.642.h5'}


In [16]:
good_models = list(center_id_to_best_model_file_name.values())
print(good_models)

['center_id_6.2.8.9_epoch_1_target_0.638.h5', 'center_id_7_epoch_3_target_0.581.h5', 'center_id_4_epoch_4_target_0.616.h5', 'center_id_1.5_epoch_4_target_0.651.h5', 'center_id_10.3_epoch_3_target_0.658.h5', 'center_id_11_epoch_2_target_0.642.h5']


In [17]:
for model_file in model_files:
    if model_file not in good_models:
        os.remove(os.path.join('/kaggle/working/models', model_file))